In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import patsy
import re

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score


Develop an algorithm to reliably estimate the value of residential houses based on fixed characteristics.
Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
Evaluate the mean dollar value of different renovations.

Perform any cleaning, feature engineering, and EDA you deem necessary.
Be sure to remove any houses that are not residential from the dataset.
Identify fixed features that can predict price.
Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
Characterize your model. How well does it perform? What are the best estimates of price?

In [2]:
house = pd.read_csv('./housing.csv')

In [3]:
house.drop(columns=["Street","Alley","LandContour","Condition1","Condition2","1stFlrSF","2ndFlrSF","FireplaceQu"],inplace=True)

In [4]:
house.groupby(by='').mean()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
MSSubClass,,,,,,,,,,,,,,,,,,,,,
20,723.998134,77.464531,11755.509328,6.033582,5.550373,1977.960821,1985.878731,104.355263,562.986940,71.958955,...,98.014925,42.811567,12.345149,5.246269,15.632463,1.208955,53.514925,6.244403,2007.875000,185224.811567
30,747.826087,60.349206,7888.492754,4.507246,5.753623,1926.637681,1968.869565,12.130435,187.826087,17.333333,...,33.927536,27.101449,61.115942,0.000000,4.144928,0.000000,35.942029,5.898551,2007.768116,95829.724638
40,725.000000,50.000000,12559.750000,5.750000,6.500000,1946.000000,1973.250000,0.000000,500.750000,0.000000,...,67.000000,7.000000,45.750000,0.000000,0.000000,0.000000,150.000000,7.750000,2008.500000,156125.000000
45,422.000000,55.000000,6949.000000,5.500000,6.166667,1934.583333,1964.000000,0.000000,103.000000,0.000000,...,4.000000,9.333333,82.416667,0.000000,0.000000,0.000000,0.000000,6.333333,2007.416667,108591.666667
50,751.194444,64.000000,10479.145833,5.402778,6.013889,1935.833333,1969.854167,34.409722,226.666667,33.347222,...,65.861111,26.583333,50.319444,4.194444,21.673611,0.000000,59.375000,6.236111,2007.840278,143302.972222
60,708.882943,81.056522,12058.076923,7.060201,5.297659,1994.494983,1996.331104,181.148649,440.525084,27.879599,...,131.638796,80.488294,8.755853,3.394649,16.933110,5.197324,18.327759,6.478261,2007.729097,239948.501672
70,771.466667,65.654545,10120.166667,6.116667,6.633333,1918.833333,1977.866667,3.733333,182.000000,6.000000,...,55.233333,36.400000,76.833333,0.000000,15.650000,0.000000,41.666667,6.950000,2007.816667,166772.416667
75,604.875000,77.533333,11631.750000,6.625000,6.250000,1912.625000,1976.750000,0.000000,188.125000,11.500000,...,52.812500,131.187500,107.312500,0.000000,33.500000,32.000000,0.000000,6.375000,2007.500000,192437.500000
80,792.155172,80.026316,10895.482759,5.965517,5.896552,1974.275862,1982.155172,128.603448,486.068966,85.810345,...,127.258621,30.827586,17.396552,3.896552,16.965517,22.655172,20.689655,6.810345,2007.810345,169736.551724
